In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd
import time
import plotly.express as px
import matplotlib.pyplot as plt
import math
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
import random
from tqdm import tqdm
from urllib.error import HTTPError


C:\Users\elija\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def update_elo(winner_elo, loser_elo, k=32, minimum=float('-inf')):
    """
    Update Elo scores after a match.

    Parameters:
    - winner_elo: Elo rating of the winner
    - loser_elo: Elo rating of the loser
    - k: Weight constant (default is 32)

    Returns:
    A tuple containing the updated Elo ratings for the winner and loser.
    """
    expected_winning_prob = 1 / (1 + 10 ** ((loser_elo - winner_elo) / 400))
    winner_new_elo = max(winner_elo + k * (1 - expected_winning_prob), minimum)
    loser_new_elo = max(loser_elo - k * expected_winning_prob, minimum)

    return winner_new_elo, loser_new_elo

In [3]:
def predict(team_elo, opp_elo):
    expected_winning_prob = 1 / (1 + 10 ** ((opp_elo - team_elo) / 400))
    return expected_winning_prob

In [4]:
years = range(2015, 2025)

In [5]:
months = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']
# months = ['april']
elos = defaultdict(lambda: 1500)

elo_over_time = pd.DataFrame()
date = None

for year in (years):

    for month in months:
        sleep = True
        try:
            month_url = f"https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html"
            df_games = pd.read_html(month_url)[0]

            for idx, game in df_games.iterrows():
                
                try:
                    if not (math.isnan(game['PTS'])):

                        dt = datetime.strptime(game['Date'], "%a, %b %d, %Y")
                        if dt != date:
                            elo_over_time.loc[date, elos.keys()] = elos.values()
                            date = dt

                        if game['PTS'] > game['PTS.1']:
                            winner = game['Visitor/Neutral']
                            loser = game['Home/Neutral']
                        else:
                            loser = game['Visitor/Neutral']
                            winner = game['Home/Neutral']

                        elos[winner], elos[loser] = update_elo(elos[winner], elos[loser])
                except(TypeError):
                    pass
                
            try:    
                del elo_over_time[None]
            except(KeyError):
                pass
            
        except(HTTPError):
            print(year, month, 'not found')
            sleep = False
            
        if sleep:
            print(year, month, 'found')
            time.sleep(4.1)

elo_over_time = elo_over_time.T

2024 october found
2024 november found
2024 december found
2024 january found
2024 february found
2024 march found
2024 april found
2024 may not found
2024 june not found


In [5]:
# THIS COLLECTS ELO RATING FOR EVERY TEAM IN THE RANGE

months = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']

season_end_elos = pd.DataFrame(columns=['year', 'team', 'elo'])

elo_over_time = pd.DataFrame()
date = None

for year in (years):
    yr_elos = defaultdict(lambda: 1500)

    for month in months:
        sleep = True
        try:
            month_url = f"https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html"
            df_games = pd.read_html(month_url)[0]

            for idx, game in df_games.iterrows():
                
                try:
                    if not (math.isnan(game['PTS'])):

                        if game['PTS'] > game['PTS.1']:
                            winner = game['Visitor/Neutral']
                            loser = game['Home/Neutral']
                        else:
                            loser = game['Visitor/Neutral']
                            winner = game['Home/Neutral']

                        yr_elos[winner], yr_elos[loser] = update_elo(yr_elos[winner], yr_elos[loser])
                except(TypeError):
                    pass
                
            try:    
                del elo_over_time[None]
            except(KeyError):
                pass
            
        except(HTTPError):
            print(year, month, 'not found')
            sleep = False
            
        if sleep:
            print(year, month, 'found')
            time.sleep(4.1)
            
    for team, elo in yr_elos.items():
        season_end_elos = season_end_elos.append({'year': year, 'team': team, 'elo': elo}, ignore_index=True)

2015 october found
2015 november found
2015 december found
2015 january found
2015 february found
2015 march found
2015 april found
2015 may found
2015 june found


C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_end_elos = season_end_elos.append({'year': year, 'team': team, 'elo': elo}, ignore_index=True)
C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_end_elos = season_end_elos.append({'year': year, 'team': team, 'elo': elo}, ignore_index=True)
C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_end_elos = season_end_elos.append({'year': year, 'team': team, 'elo': elo}, ignore_index=True)
C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The fr

2016 october found
2016 november found
2016 december found
2016 january found
2016 february found
2016 march found
2016 april found
2016 may found
2016 june found


C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_end_elos = season_end_elos.append({'year': year, 'team': team, 'elo': elo}, ignore_index=True)
C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_end_elos = season_end_elos.append({'year': year, 'team': team, 'elo': elo}, ignore_index=True)
C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_end_elos = season_end_elos.append({'year': year, 'team': team, 'elo': elo}, ignore_index=True)
C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The fr

2017 october found
2017 november found
2017 december found
2017 january found
2017 february found
2017 march found
2017 april found
2017 may found
2017 june found


...

2018 october found
2018 november found
2018 december found
2018 january found
2018 february found
2018 march found
2018 april found
2018 may found
2018 june found


C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_end_elos = season_end_elos.append({'year': year, 'team': team, 'elo': elo}, ignore_index=True)
C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_end_elos = season_end_elos.append({'year': year, 'team': team, 'elo': elo}, ignore_index=True)
C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_end_elos = season_end_elos.append({'year': year, 'team': team, 'elo': elo}, ignore_index=True)
C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The fr

2019 october found
2019 november found
2019 december found
2019 january found
2019 february found
2019 march found
2019 april found
2019 may found
2019 june found


C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_end_elos = season_end_elos.append({'year': year, 'team': team, 'elo': elo}, ignore_index=True)
C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_end_elos = season_end_elos.append({'year': year, 'team': team, 'elo': elo}, ignore_index=True)
C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_end_elos = season_end_elos.append({'year': year, 'team': team, 'elo': elo}, ignore_index=True)
C:\Users\elija\AppData\Local\Temp\ipykernel_3464\1541966574.py:49: FutureWarning: The fr

ValueError: No tables found

In [6]:
team_colors = {
    "Atlanta Hawks": "#E03A3E",
    "Boston Celtics": "#008348",
    "Brooklyn Nets": "#000000",
    "Charlotte Hornets": "#00788C",
    "Chicago Bulls": "#CE1141",
    "Cleveland Cavaliers": "#860038",
    "Dallas Mavericks": "#00538C",
    "Denver Nuggets": "#0E2240",
    "Detroit Pistons": "#C8102E",
    "Golden State Warriors": "#006BB6",
    "Houston Rockets": "#CE1141",
    "Indiana Pacers": "#002D62",
    "Los Angeles Clippers": "#1D428A",
    "Los Angeles Lakers": "#552583",
    "Memphis Grizzlies": "#5D76A9",
    "Miami Heat": "#98002E",
    "Milwaukee Bucks": "#00471B",
    "Minnesota Timberwolves": "#0C2340",
    "New Orleans Pelicans": "#0C2340",
    "New York Knicks": "#F58426",
    "Oklahoma City Thunder": "#007AC1",
    "Orlando Magic": "#0077C0",
    "Philadelphia 76ers": "#006BB6",
    "Phoenix Suns": "#1D1160",
    "Portland Trail Blazers": "#E03A3E",
    "Sacramento Kings": "#5A2D81",
    "San Antonio Spurs": "#C4CED4",
    "Toronto Raptors": "#CE1141",
    "Utah Jazz": "#002B5C",
    "Washington Wizards": "#E31837",
}

team_conferences = {
    "Atlanta Hawks": "Eastern Conference",
    "Boston Celtics": "Eastern Conference",
    "Brooklyn Nets": "Eastern Conference",
    "Charlotte Hornets": "Eastern Conference",
    "Chicago Bulls": "Eastern Conference",
    "Cleveland Cavaliers": "Eastern Conference",
    "Dallas Mavericks": "Western Conference",
    "Denver Nuggets": "Western Conference",
    "Detroit Pistons": "Eastern Conference",
    "Golden State Warriors": "Western Conference",
    "Houston Rockets": "Western Conference",
    "Indiana Pacers": "Eastern Conference",
    "Los Angeles Clippers": "Western Conference",
    "Los Angeles Lakers": "Western Conference",
    "Memphis Grizzlies": "Western Conference",
    "Miami Heat": "Eastern Conference",
    "Milwaukee Bucks": "Eastern Conference",
    "Minnesota Timberwolves": "Western Conference",
    "New Orleans Pelicans": "Western Conference",
    "New York Knicks": "Eastern Conference",
    "Oklahoma City Thunder": "Western Conference",
    "Orlando Magic": "Eastern Conference",
    "Philadelphia 76ers": "Eastern Conference",
    "Phoenix Suns": "Western Conference",
    "Portland Trail Blazers": "Western Conference",
    "Sacramento Kings": "Western Conference",
    "San Antonio Spurs": "Western Conference",
    "Toronto Raptors": "Eastern Conference",
    "Utah Jazz": "Western Conference",
    "Washington Wizards": "Eastern Conference",
}

In [7]:
predict(elos['Boston Celtics'], elos['Los Angeles Lakers'])

0.8066599036013001

## Elos work. Now let's predict the season. 

In [8]:
def series_win_probability(single_game_probability):
    single_game_probability = max(0, min(1, single_game_probability))

    # Calculate the probability of winning the series
    series_probability = 0
    for i in range(4, 8):
        series_probability += (single_game_probability ** i) * ((1 - single_game_probability) ** (7 - i)) * math.comb(7, i)

    return series_probability

game_prob = predict(elos['Boston Celtics'], elos['Los Angeles Lakers'])
result = series_win_probability(game_prob)
print(f"The probability of winning the series is: {result:.2%}")

The probability of winning the series is: 97.03%


## But first, let's create a master df to make this easier

In [9]:
df_elo = pd.DataFrame(pd.Series(elos), columns=['Elo'])
df_elo.head()

,Elo
Denver Nuggets,1623.922196
Los Angeles Lakers,1469.192910
Phoenix Suns,1488.839770
Golden State Warriors,1384.003429
Orlando Magic,1490.067078


In [10]:
df_conf = pd.DataFrame(pd.Series(team_conferences), columns=['Conf'])
df_color = pd.DataFrame(pd.Series(team_colors), columns=['Color'])
df_master = df_elo.join(df_color).join(df_conf)
df_master = df_master.sort_values('Elo', ascending=False)
df_master = df_master.join(elo_over_time)
df_master.head()

,Elo,Color,Conf,NaT,2023-10-24 00:00:00,2023-10-25 00:00:00,2023-10-26 00:00:00,2023-10-27 00:00:00,2023-10-28 00:00:00,2023-10-29 00:00:00,...,2024-01-02 00:00:00,2024-01-03 00:00:00,2024-01-04 00:00:00,2024-01-05 00:00:00,2024-01-06 00:00:00,2024-01-07 00:00:00,2024-01-08 00:00:00,2024-01-09 00:00:00,2024-01-10 00:00:00,2024-01-11 00:00:00
Boston Celtics,1717.340326,#008348,Eastern Conference,NaN,NaN,1516.0,1516.0,1532.000000,1532.000000,1532.000000,...,1708.774589,1708.774589,1708.774589,1713.129965,1723.287030,1723.287030,1714.362168,1714.362168,1728.885147,1717.340326
Oklahoma City Thunder,1680.648252,#007AC1,Western Conference,NaN,NaN,1516.0,1516.0,1532.000000,1532.000000,1516.067673,...,1674.853246,1671.071832,1671.071832,1666.588233,1666.588233,1666.588233,1667.002145,1667.002145,1678.853966,1680.648252
Minnesota Timberwolves,1666.364353,#0C2340,Western Conference,NaN,NaN,1484.0,1484.0,1484.000000,1500.736307,1500.736307,...,1684.524286,1672.708821,1672.708821,1681.369307,1681.369307,1672.395618,1672.395618,1682.197500,1664.720480,1664.720480
Milwaukee Bucks,1649.973908,#00471B,Eastern Conference,NaN,NaN,NaN,1516.0,1516.000000,1516.000000,1502.196525,...,1655.291598,1643.894120,1644.500035,1644.500035,1635.661361,1635.661361,1629.518730,1629.518730,1629.518730,1649.973908
Los Angeles Clippers,1626.833176,#1D428A,Western Conference,NaN,NaN,1516.0,1516.0,1501.469502,1501.469502,1517.401829,...,1587.062323,1599.070553,1599.070553,1615.651061,1615.651061,1606.723552,1617.678839,1617.678839,1622.973747,1622.973747


In [31]:
fig = px.bar(df_master,
    x=df_master.index,
    y='Elo',
    labels={'index': 'Team',},
    title='NBA Team Elo',
    color=df_master.index,
    color_discrete_sequence=df_master['Color'],

)

fig.update_layout(showlegend=False)

# Show the plot
fig.show()

In [12]:
fig = make_subplots(rows=2, cols=1, subplot_titles=['Eastern Conference', 'Western Conference'])

df_east = df_master[df_master['Conf'] == 'Eastern Conference']

# Add bars for Eastern Conference
east_bar = go.Bar(
    x=df_east.index,
    y=df_east['Elo'],
    marker=dict(color=df_east['Color']),
    showlegend = False
)
fig.add_trace(east_bar, row=1, col=1)

df_west = df_master[df_master['Conf'] == 'Western Conference']
west_bar = go.Bar(
    x=df_west.index,
    y=df_west['Elo'],
    marker=dict(color=df_west['Color']),
    showlegend=False
)

fig.add_trace(west_bar, row=2, col=1)

fig.add_shape(
    type="line",
    x0=5.5,
    x1=5.5,
    y0=.625,
    y1=1,
    xref="x",
    yref="paper",
    line=dict(color="black", width=2, dash='dash'),
)

fig.add_shape(
    type="line",
    x0=9.5,
    x1=9.5,
    y0=.625,
    y1=1,
    xref="x",
    yref="paper",
    line=dict(color="black", width=2),
)

fig.add_shape(
    type="line",
    x0=5.5,
    x1=5.5,
    y0=0,
    y1=.375,
    xref="x",
    yref="paper",
    line=dict(color="black", width=2, dash='dash'),
)

fig.add_shape(
    type="line",
    x0=9.5,
    x1=9.5,
    y0=0,
    y1=.375,
    xref="x",
    yref="paper",
    line=dict(color="black", width=2),
)

fig.update_layout(
    title_text='NBA Conference Elo Ratings',  # Add an overall title if needed
    margin=dict(l=20, r=20, t=50, b=50),  # Adjust margin around the plot
    height=800,  # Adjust the height of the entire figure
    width=800
)

In [13]:
def remove_seed(input_string):
    star_index = input_string.rfind('*')
    
    # If an opening parenthesis is found, remove it and the following characters
    if star_index != -1:
        result_string = input_string[:star_index].strip()
    else:
        parenthesis_index = input_string.rfind('(')
        # If an opening parenthesis is found, remove it and the following characters
        if parenthesis_index != -1:
            result_string = input_string[:parenthesis_index].strip()
        else:
            # If no opening parenthesis is found, return the original string
            result_string = input_string
    
    return result_string

In [14]:
standings = pd.read_html(f'https://www.basketball-reference.com/leagues/NBA_{year}_standings.html')
time.sleep(4.1)

e = standings[0]
try:
    e['Eastern Conference'] = e['Eastern Conference'].apply(remove_seed)
    e.set_index('Eastern Conference', inplace=True)
except(KeyError):
    pass

w = standings[1]
try:
    w['Western Conference'] = w['Western Conference'].apply(remove_seed)
    w.set_index('Western Conference', inplace=True)
except(KeyError):
    pass

df_master = df_master.join(pd.concat([e, w]))
df_master.head()

,Elo,Color,Conf,NaT,2023-10-24 00:00:00,2023-10-25 00:00:00,2023-10-26 00:00:00,2023-10-27 00:00:00,2023-10-28 00:00:00,2023-10-29 00:00:00,...,2024-01-09 00:00:00,2024-01-10 00:00:00,2024-01-11 00:00:00,W,L,W/L%,GB,PS/G,PA/G,SRS
Boston Celtics,1717.340326,#008348,Eastern Conference,NaN,NaN,1516.0,1516.0,1532.000000,1532.000000,1532.000000,...,1714.362168,1728.885147,1717.340326,29,9,0.763,—,120.8,111.6,9.92
Oklahoma City Thunder,1680.648252,#007AC1,Western Conference,NaN,NaN,1516.0,1516.0,1532.000000,1532.000000,1516.067673,...,1667.002145,1678.853966,1680.648252,26,11,0.703,0.5,122.8,113.7,9.06
Minnesota Timberwolves,1666.364353,#0C2340,Western Conference,NaN,NaN,1484.0,1484.0,1484.000000,1500.736307,1500.736307,...,1682.197500,1664.720480,1664.720480,27,11,0.711,—,113.2,107.4,6.84
Milwaukee Bucks,1649.973908,#00471B,Eastern Conference,NaN,NaN,NaN,1516.0,1516.000000,1516.000000,1502.196525,...,1629.518730,1629.518730,1649.973908,26,12,0.684,3.0,124.4,119.4,4.10
Los Angeles Clippers,1626.833176,#1D428A,Western Conference,NaN,NaN,1516.0,1516.0,1501.469502,1501.469502,1517.401829,...,1617.678839,1622.973747,1622.973747,25,13,0.658,2.0,117.7,111.9,5.25


In [15]:
fig = px.scatter(df_master, x='Elo', y='SRS', hover_name=df_master.index, hover_data=['W/L%'], 
#                  symbol='Conf',
           color_discrete_sequence=df_master['Color'], color=df_master.index, 
          title='Comparing Elos to SRS')

fig.add_shape(
    type='line',
    x0=df_master['Elo'].min(),
    x1=df_master['Elo'].max(),
    y0=0,
    y1=0,
    line=dict(color='gray', width=1)
)

# fig.add_shape(
#     type='line',
#     x0=df_master['Elo'].mean(),
#     x1=df_master['Elo'].mean(),
#     y0=df_master['SRS'].min(),
#     y1=df_master['SRS'].max(),
#     line=dict(color='gray', width=1)
# )

fig.update_layout(showlegend=False)
fig.show()

In [16]:
from sklearn.linear_model import LinearRegression

X = df_master['Elo'].values.reshape(-1, 1)
y = df_master['W/L%'].values.reshape(-1, 1)


lr = LinearRegression()
lr.fit(X, y)

LinearRegression()

In [17]:
from sklearn.preprocessing import PolynomialFeatures

# Create quadratic features
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

# Fit quadratic regression model
lr_quad = LinearRegression()
lr_quad.fit(X_poly, y)

lr_quad.coef_

array([[ 0.00000000e+00, -2.34018334e-04,  3.59004525e-07]])

In [18]:
fig = px.scatter(df_master, x='Elo', y='W/L%', hover_name=df_master.index, hover_data=['SRS', 'Conf'], 
        color_discrete_sequence=df_master['Color'], color=df_master.index, 
          title='Comparing Elos to Win Percentage')

x_range = np.linspace(df_master['Elo'].min(), df_master['Elo'].max(), 100).reshape(-1, 1)
y_quad_line = lr_quad.predict(poly.transform(x_range))

fig.add_trace(go.Scatter(x=x_range.flatten(), y=y_quad_line.flatten(), mode='lines', 
                         line=dict(color='gray', width=1), showlegend=False))

fig.update_layout(showlegend=False)
fig.show()


## Working with moneylines

In [19]:
def ml_to_prob(moneyline):
    if moneyline > 0:
        probability = 100 / (moneyline + 100)
    elif moneyline < 0:
        probability = abs(moneyline) / (abs(moneyline) + 100)
    else:
        probability = 0.5  # Even odds

    return probability

In [20]:
moneyline_odds = -1450
probability = ml_to_prob(moneyline_odds)
print(f"The probability corresponding to the moneyline odds {moneyline_odds} is: {probability:.2%}")

The probability corresponding to the moneyline odds -1450 is: 93.55%


In [21]:
predict(elos['Sacramento Kings'], elos['Charlotte Hornets'])

0.9537494835756419

## More graphs bc why not

In [30]:
fig = px.line(df_master[df_master.columns[3:-7]].T, color_discrete_sequence=df_master.Color, 
       labels={'variable':'Team', 
               'value': 'Elo', 
               'index':'Date'
           }, 
        title='NBA Elos over time'
       )

fig.write_html("1-13-elo.html")
fig.show()

In [23]:
px.line(df_east[df_east.columns[3:]].T, color_discrete_sequence=df_east.Color, 
       labels={'variable':'Team', 
               'value': 'ELO', 
               'index':'Date'
           }
       )

In [24]:
# there's something here but it'll take some time to get right. might have to use go
px.violin(df_master.T[30:-7], color_discrete_sequence=df_master.Color)

In [25]:
# once this stablizes, the ELO's are about accurate
px.violin(df_master.T[30:-7].T, points='all')

In [27]:
# Assuming x is the desired number of top positions
top = 5

days_in_top = defaultdict(lambda: 0)

for date in df_master.columns[10:-7]:

    df_temp = df_master.sort_values(date, ascending=False)

    top_indices = df_temp.index[:top].tolist()

    for idx in top_indices:
        days_in_top[idx] += 1

df_days_in_top = pd.DataFrame(days_in_top, index=[f'Days in Top {top}']).T.sort_values(f'Days in Top {top}', ascending=False)

for idx in df_days_in_top.index:
    df_days_in_top.loc[idx, 'Color'] = team_colors[idx]
